In [28]:
# simple collision detection
import pygame

pygame.init()

screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Sprites Example")

speed = 5

#tickrate
clock = pygame.time.Clock()

# Load images
player_img = pygame.image.load("zelda.png")  # Use any image
player_img = pygame.transform.scale(player_img, (50, 50))  # Resize
enemy_img = pygame.image.load("ganon.png")
enemy_img = pygame.transform.scale(enemy_img, (100, 100))

# Load background image
background_img = pygame.image.load("white.png")
background_img = pygame.transform.scale(background_img, (800, 600))

player_pos = [100, 100]
enemy_pos = [300, 300]

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_pos[0] -= speed
    if keys[pygame.K_RIGHT]:
        player_pos[0] += speed
    if keys[pygame.K_UP]:
        player_pos[1] -= speed
    if keys[pygame.K_DOWN]:
        player_pos[1] += speed

    # Collision detection
    player_rect = pygame.Rect(*player_pos, 50, 50)
    enemy_rect = pygame.Rect(*enemy_pos, 100, 100)
    collision = player_rect.colliderect(enemy_rect)
    

    # Draw background image
    screen.blit(background_img, (0, 0))
    screen.blit(player_img, player_pos)
    screen.blit(enemy_img, enemy_pos)

    # Draw green rectangles around player and enemy
    pygame.draw.rect(screen, (0, 255, 0), player_rect, 3)
    pygame.draw.rect(screen, (0, 255, 0), enemy_rect, 3)
    
    if collision:
        pygame.draw.rect(screen, (255, 0, 0), player_rect, 3)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


In [26]:
# aside: list comprehensions - intro when they see for first time
#### new_list = [expression for member in iterable if conditional]
# FILTER ENTRIES
numbers = [1, 2, 3, 4, 5, 6]
# getting even numbers
evens = []
for number in numbers:
    if number % 2 == 0:
        evens.append(number)
print(evens)
# shorter way
evens2 = [number for number in numbers if number % 2 == 0]
print(evens2)

# first 10 squares: conditional is optional
squares = []
for num in range(10):
    squares.append(num * num)

# exercise: convert to list comprehension. Solution below:
squares2 = [number * number for number in range(10)]
print(squares2)

##########################################
### new_list = [true_expr if conditional else false_expr for member in iterable]
# MODIFY ENTRIES
original_prices = [1.25, -9.45, 10.22, 3.78, -5.92, 1.16]
no_negatives = [price if price > 0 else 0 for price in original_prices]
print(no_negatives)

### combining both 
even_sq = [num*num if num % 2 == 0 else num for num in numbers if num % 3 == 0]
print(even_sq)

### you can also do set and dictionary comprehensions
squares2 = {number: number * number for number in range(10)}

# pro: shorter code. con: sometimes slower. 

[2, 4, 6]
[2, 4, 6]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
[1.25, 0, 10.22, 3.78, 0, 1.16]
[3, 36]


In [32]:
# aside: unpacking lists
def greet(name, age):
    print(f"Hello, {name}. You are {age} years old.")

info = ["Alice", 30]
greet(*info)

Hello, Alice. You are 30 years old.


In [4]:
# pixel perfect collision detection
import pygame

pygame.init()

screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Advanced Collision Detection")

speed = 5

#tickrate
clock = pygame.time.Clock()

# Load images
player_img = pygame.image.load("zelda.png").convert_alpha()  # Use any image
player_img = pygame.transform.scale(player_img, (50, 50))  # Resize
enemy_img = pygame.image.load("ganon.png").convert_alpha()
enemy_img = pygame.transform.scale(enemy_img, (100, 100))

# Create masks
player_mask = pygame.mask.from_surface(player_img)
enemy_mask = pygame.mask.from_surface(enemy_img)

# Load background image
background_img = pygame.image.load("white.png")
background_img = pygame.transform.scale(background_img, (800, 600))

player_pos = [100, 100]
enemy_pos = [300, 300]

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_pos[0] -= speed
    if keys[pygame.K_RIGHT]:
        player_pos[0] += speed
    if keys[pygame.K_UP]:
        player_pos[1] -= speed
    if keys[pygame.K_DOWN]:
        player_pos[1] += speed

    # Collision detection using masks
    player_rect = pygame.Rect(*player_pos, 50, 50)
    enemy_rect = pygame.Rect(*enemy_pos, 100, 100)
    offset = (enemy_rect.left - player_rect.left, enemy_rect.top - player_rect.top)
    collision = player_mask.overlap(enemy_mask, offset)

    # Draw background image
    screen.blit(background_img, (0, 0))
    screen.blit(player_img, player_pos)
    screen.blit(enemy_img, enemy_pos)
    
    # Draw the outline of the player image
    player_outline = player_mask.outline()
    player_outline = [(x + player_pos[0], y + player_pos[1]) for x, y in player_outline]
    pygame.draw.lines(screen, (0, 255, 0), True, player_outline, 3)
    
    # Draw the outline of the enemy image
    enemy_outline = enemy_mask.outline()
    enemy_outline = [(x + enemy_pos[0], y + enemy_pos[1]) for x, y in enemy_outline]
    pygame.draw.lines(screen, (0, 255, 0), True, enemy_outline, 3)
    
    if collision:
        pygame.draw.lines(screen, (255, 0, 0), True, player_outline, 3)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

In [29]:
# add enemy movement, timer, lives
import pygame
import sys
import random

pygame.init()

screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Avoid the Enemy")

speed = 5
enemy_speed = 3
lives = 3
game_duration = 30 * 1000  # 30 seconds in milliseconds
start_time = pygame.time.get_ticks()

PLAYER_WIDTH = 50
PLAYER_HEIGHT = 50

# Tickrate
clock = pygame.time.Clock()

# Load images
player_img = pygame.image.load("zelda.png").convert_alpha()
player_img = pygame.transform.scale(player_img, (PLAYER_WIDTH, PLAYER_HEIGHT))
enemy_img = pygame.image.load("ganon.png").convert_alpha()
enemy_img = pygame.transform.scale(enemy_img, (100, 100))

# Create masks
player_mask = pygame.mask.from_surface(player_img)
enemy_mask = pygame.mask.from_surface(enemy_img)

# Load background image
background_img = pygame.image.load("white.png")
background_img = pygame.transform.scale(background_img, (800, 600))

player_pos = [100, 100]
enemy_pos = [300, 300]

font = pygame.font.SysFont("Arial", 30)

# Enemy movement direction
enemy_direction = [random.choice([-1, 1]), random.choice([-1, 1])]

running = True
game_over = False
while running:
    while game_over:
        # Game Over or Win Screen
        screen.fill((255, 255, 255))
        if lives > 0:
            result_text = font.render("You Win! Press 'C' to play again or 'Q' to quit.", True, (0, 255, 0))
        else:
            result_text = font.render("Game Over! Press 'C' to play again or 'Q' to quit.", True, (255, 0, 0))
        final_score_text = font.render(f"Lives Remaining: {lives}", True, (0, 0, 0))
        screen.blit(result_text, (screen.get_width() // 2 - result_text.get_width() // 2, 200))
        screen.blit(final_score_text, (screen.get_width() // 2 - final_score_text.get_width() // 2, 250))
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                    game_over = False
                if event.key == pygame.K_c:
                    game_over = False
                    player_pos = [100, 100]
                    enemy_pos = [300, 300]
                    lives = 3
                    start_time = pygame.time.get_ticks()
                    
    elapsed_time = pygame.time.get_ticks() - start_time
    remaining_time = max(0, (game_duration - elapsed_time) // 1000)  # Convert to seconds

    if remaining_time == 0 or lives == 0:
        game_over = True

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_pos[0] -= speed
    if keys[pygame.K_RIGHT]:
        player_pos[0] += speed
    if keys[pygame.K_UP]:
        player_pos[1] -= speed
    if keys[pygame.K_DOWN]:
        player_pos[1] += speed

    # Move the enemy
    enemy_pos[0] += enemy_direction[0] * enemy_speed
    enemy_pos[1] += enemy_direction[1] * enemy_speed

    # Bounce the enemy off the screen edges
    if enemy_pos[0] <= 0 or enemy_pos[0] + 100 >= 800:
        enemy_direction[0] *= -1
    if enemy_pos[1] <= 0 or enemy_pos[1] + 100 >= 600:
        enemy_direction[1] *= -1

    # Limit player to the window
    if player_pos[0] < 0:
        player_pos[0] = 0
    elif player_pos[0] > 800 - PLAYER_WIDTH:
        player_pos[0] = 800 - PLAYER_WIDTH
    if player_pos[1] < 0:
        player_pos[1] = 0
    elif player_pos[1] > 600 - PLAYER_HEIGHT:
        player_pos[1] = 600 - PLAYER_HEIGHT
        

    # Collision detection using masks
    player_rect = pygame.Rect(*player_pos, 50, 50)
    enemy_rect = pygame.Rect(*enemy_pos, 100, 100)
    offset = (enemy_rect.left - player_rect.left, enemy_rect.top - player_rect.top)
    collision = player_mask.overlap(enemy_mask, offset)

    if collision:
        lives -= 1
        pygame.draw.lines(screen, (255, 0, 0), True, player_outline, 3) # draw red player outline
        screen.blit(font.render("Ganon hit you!", True, (255, 0, 0)), player_pos)
        pygame.display.flip()
        pygame.time.wait(1000)
        start_time += 1000
        # player_pos = [100, 100]  # Reset player position
        player_pos = [random.randint(0,800), random.randint(0,600)]  # respawn player in random position on screen

    # Draw background image
    screen.blit(background_img, (0, 0))
    screen.blit(player_img, player_pos)
    screen.blit(enemy_img, enemy_pos)

    # Draw the outline of the player image
    player_outline = player_mask.outline()
    player_outline = [(x + player_pos[0], y + player_pos[1]) for x, y in player_outline]
    pygame.draw.lines(screen, (0, 255, 0), True, player_outline, 3)

    # Draw the outline of the enemy image
    enemy_outline = enemy_mask.outline()
    enemy_outline = [(x + enemy_pos[0], y + enemy_pos[1]) for x, y in enemy_outline]
    pygame.draw.lines(screen, (0, 255, 0), True, enemy_outline, 3)

    # if collision:
    #     pygame.draw.lines(screen, (255, 0, 0), True, player_outline, 3)

    # Display lives and time
    lives_text = font.render(f"Lives: {lives}", True, (0, 0, 0))
    time_text = font.render(f"Time: {remaining_time}s", True, (0, 0, 0))
    screen.blit(lives_text, (10, 10))
    screen.blit(time_text, (10, 50))

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

SystemExit: 

### Unpacking lists
**Where it is in your code**: 
```python
player_rect = pygame.Rect(*player_pos, 50, 50)
```
**What it is**:

A way of passing the elements of lists (or tuples, sets, dictionaries) to functions individually, *not* as a whole list. For example:
```python
def greet(name, age):
    print(f"Hello, {name}. You are {age} years old.")

info = ["Alice", 30]
greet(*info)
```
The function `greet()` has two parameters, `name` and `age`. If we write `greet(info)` without the `*`, Python would raise an error, because you are passing only a single argument, `info`, which is a list. However, by writing `greet(*info)`, you tell Python to *unpack* the list `info`: give the individual entries of `info` to the function.

**Exercise**
Use the `*` operator to sum the entries of `numbers` using the function below.
```python
numbers = [4, 7, 1]

def sum_three_numbers(a, b, c):
    return sum(a, b, c)

result = ...
```

### List comprehensions
**Where it is in your code**: 
```python
player_outline = [(x + player_pos[0], y + player_pos[1]) for x, y in player_outline]
```
**What it is**:

Say you have a list of numbers.
```python
numbers = [1, 2, 3, 4, 5, 6]
```
If you want to get all the even numbers from the list, one way is to create an empty list and then go through each entry of the original list using a for loop, check if it is even, and add it to the new list:
```python
evens = []
for num in numbers:
    if num % 2 == 0:
        evens.append(num)
```
A shorter way to write this is using a *list comprehension*. The code above could be shortened to: 
```python
evens = [num for num in numbers if num % 2 == 0]
```
Notice how the syntax is similar to how you would write the code without using comprehensions. The general form is:
```python
new_list = [expression for member in iterable if conditional]
```
**Advantages**: list comprehensions can sometimes make your code shorter and easier to write and read.

**Disadvantages**: list comprehensions might sometimes run more slowly than other methods of writing your code.

**Exercise**: write the following as a list comprehension.
```python
squares = []
for num in range(10):
    squares.append(num * num)
```
**Exercise**: what's the long way to write the `player_outline` code? (answer below)
```python
player_outline = player_mask.outline()
new_list = []
for x, y in player_outline:
    new_list.append(x + player_pos[0], y + player_pos[1])
```

In [2]:
# add music and collision sound
import pygame
import sys
import random

pygame.init()
pygame.mixer.init()

screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Avoid the Enemy")

pygame.mixer.music.load("riffusion-eclipse.mp3")
pygame.mixer.music.play(-1)  # Loop the music

collision_sound = pygame.mixer.Sound("collide.mp3")

speed = 5
enemy_speed = 3
lives = 3
game_duration = 30 * 1000  # 30 seconds in milliseconds
start_time = pygame.time.get_ticks()

PLAYER_WIDTH = 50
PLAYER_HEIGHT = 50

# Tickrate
clock = pygame.time.Clock()

# Load images
player_img = pygame.image.load("zelda.png").convert_alpha()
player_img = pygame.transform.scale(player_img, (PLAYER_WIDTH, PLAYER_HEIGHT))
enemy_img = pygame.image.load("ganon.png").convert_alpha()
enemy_img = pygame.transform.scale(enemy_img, (100, 100))

# Create masks
player_mask = pygame.mask.from_surface(player_img)
enemy_mask = pygame.mask.from_surface(enemy_img)

# Load background image
background_img = pygame.image.load("white.png")
background_img = pygame.transform.scale(background_img, (800, 600))

player_pos = [100, 100]
enemy_pos = [300, 300]

font = pygame.font.SysFont("Arial", 30)

# Enemy movement direction
enemy_direction = [random.choice([-1, 1]), random.choice([-1, 1])]

running = True
game_over = False
while running:
    while game_over:
        # Game Over or Win Screen
        screen.fill((255, 255, 255))
        if lives > 0:
            result_text = font.render("You Win! Press 'C' to play again or 'Q' to quit.", True, (0, 255, 0))
        else:
            result_text = font.render("Game Over! Press 'C' to play again or 'Q' to quit.", True, (255, 0, 0))
        final_score_text = font.render(f"Lives Remaining: {lives}", True, (0, 0, 0))
        screen.blit(result_text, (screen.get_width() // 2 - result_text.get_width() // 2, 200))
        screen.blit(final_score_text, (screen.get_width() // 2 - final_score_text.get_width() // 2, 250))
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                    game_over = False
                if event.key == pygame.K_c:
                    game_over = False
                    player_pos = [100, 100]
                    enemy_pos = [300, 300]
                    lives = 3
                    start_time = pygame.time.get_ticks()
                    
    elapsed_time = pygame.time.get_ticks() - start_time
    remaining_time = max(0, (game_duration - elapsed_time) // 1000)  # Convert to seconds

    if remaining_time == 0 or lives == 0:
        game_over = True

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_pos[0] -= speed
    if keys[pygame.K_RIGHT]:
        player_pos[0] += speed
    if keys[pygame.K_UP]:
        player_pos[1] -= speed
    if keys[pygame.K_DOWN]:
        player_pos[1] += speed

    # Move the enemy
    enemy_pos[0] += enemy_direction[0] * enemy_speed
    enemy_pos[1] += enemy_direction[1] * enemy_speed

    # Bounce the enemy off the screen edges
    if enemy_pos[0] <= 0 or enemy_pos[0] + 100 >= 800:
        enemy_direction[0] *= -1
    if enemy_pos[1] <= 0 or enemy_pos[1] + 100 >= 600:
        enemy_direction[1] *= -1

    # Limit player to the window
    if player_pos[0] < 0:
        player_pos[0] = 0
    elif player_pos[0] > 800 - PLAYER_WIDTH:
        player_pos[0] = 800 - PLAYER_WIDTH
    if player_pos[1] < 0:
        player_pos[1] = 0
    elif player_pos[1] > 600 - PLAYER_HEIGHT:
        player_pos[1] = 600 - PLAYER_HEIGHT
        

    # Collision detection using masks
    player_rect = pygame.Rect(*player_pos, 50, 50)
    enemy_rect = pygame.Rect(*enemy_pos, 100, 100)
    offset = (enemy_rect.left - player_rect.left, enemy_rect.top - player_rect.top)
    collision = player_mask.overlap(enemy_mask, offset)

    if collision:
        lives -= 1
        collision_sound.play()
        pygame.draw.lines(screen, (255, 0, 0), True, player_outline, 3) # draw red player outline
        screen.blit(font.render("Ganon hit you!", True, (255, 0, 0)), player_pos)
        pygame.display.flip()
        pygame.time.wait(1000)
        start_time += 1000
        # player_pos = [100, 100]  # Reset player position
        player_pos = [random.randint(0,800), random.randint(0,600)]  # respawn player in random position on screen

    # Draw background image
    screen.blit(background_img, (0, 0))
    screen.blit(player_img, player_pos)
    screen.blit(enemy_img, enemy_pos)

    # Draw the outline of the player image
    player_outline = player_mask.outline()
    player_outline = [(x + player_pos[0], y + player_pos[1]) for x, y in player_outline]
    pygame.draw.lines(screen, (0, 255, 0), True, player_outline, 3)

    # Draw the outline of the enemy image
    enemy_outline = enemy_mask.outline()
    enemy_outline = [(x + enemy_pos[0], y + enemy_pos[1]) for x, y in enemy_outline]
    pygame.draw.lines(screen, (0, 255, 0), True, enemy_outline, 3)

    # if collision:
    #     pygame.draw.lines(screen, (255, 0, 0), True, player_outline, 3)

    # Display lives and time
    lives_text = font.render(f"Lives: {lives}", True, (0, 0, 0))
    time_text = font.render(f"Time: {remaining_time}s", True, (0, 0, 0))
    screen.blit(lives_text, (10, 10))
    screen.blit(time_text, (10, 50))

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

SystemExit: 

In [1]:
# add animation
import pygame
import sys
import random

pygame.init()
pygame.mixer.init()

screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Avoid the Enemy")

pygame.mixer.music.load("riffusion-eclipse.mp3")
pygame.mixer.music.play(-1)  # Loop the music

collision_sound = pygame.mixer.Sound("collide.mp3")

speed = 5
enemy_speed = 3
lives = 3
game_duration = 30 * 1000  # 30 seconds in milliseconds
start_time = pygame.time.get_ticks()

PLAYER_WIDTH = 80
PLAYER_HEIGHT = 80

# Tickrate
clock = pygame.time.Clock()

# Load images
# player_img = pygame.image.load("zelda.png").convert_alpha()
# player_img = pygame.transform.scale(player_img, (PLAYER_WIDTH, PLAYER_HEIGHT))
enemy_img = pygame.image.load("ganon.png").convert_alpha()
enemy_img = pygame.transform.scale(enemy_img, (100, 100))

# load sprites for animation # CHANGED
player_sprites = [pygame.image.load(f"Sprites/sprite{i}.png").convert_alpha() for i in range(1, 21)]
player_sprites = [pygame.transform.scale(sprite, (PLAYER_WIDTH, PLAYER_HEIGHT)) for sprite in player_sprites]
player_index = 0

# Create masks
# player_mask = pygame.mask.from_surface(player_img) # CHANGED
enemy_mask = pygame.mask.from_surface(enemy_img)

# Load background image
background_img = pygame.image.load("white.png")
background_img = pygame.transform.scale(background_img, (800, 600))

player_pos = [100, 100]
enemy_pos = [300, 300]

font = pygame.font.SysFont("Arial", 30)

# Enemy movement direction
enemy_direction = [random.choice([-1, 1]), random.choice([-1, 1])]

running = True
game_over = False
while running:
    while game_over:
        # Game Over or Win Screen
        screen.fill((255, 255, 255))
        if lives > 0:
            result_text = font.render("You Win! Press 'C' to play again or 'Q' to quit.", True, (0, 255, 0))
        else:
            result_text = font.render("Game Over! Press 'C' to play again or 'Q' to quit.", True, (255, 0, 0))
        final_score_text = font.render(f"Lives Remaining: {lives}", True, (0, 0, 0))
        screen.blit(result_text, (screen.get_width() // 2 - result_text.get_width() // 2, 200))
        screen.blit(final_score_text, (screen.get_width() // 2 - final_score_text.get_width() // 2, 250))
        pygame.display.flip()
        
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    running = False
                    game_over = False
                if event.key == pygame.K_c:
                    game_over = False
                    player_pos = [100, 100]
                    enemy_pos = [300, 300]
                    lives = 3
                    start_time = pygame.time.get_ticks()
                    
    elapsed_time = pygame.time.get_ticks() - start_time
    remaining_time = max(0, (game_duration - elapsed_time) // 1000)  # Convert to seconds

    if remaining_time == 0 or lives == 0:
        game_over = True

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player_pos[0] -= speed
    if keys[pygame.K_RIGHT]:
        player_pos[0] += speed
    if keys[pygame.K_UP]:
        player_pos[1] -= speed
    if keys[pygame.K_DOWN]:
        player_pos[1] += speed

    # Move the enemy
    enemy_pos[0] += enemy_direction[0] * enemy_speed
    enemy_pos[1] += enemy_direction[1] * enemy_speed

    # Bounce the enemy off the screen edges
    if enemy_pos[0] <= 0 or enemy_pos[0] + 100 >= 800:
        enemy_direction[0] *= -1
    if enemy_pos[1] <= 0 or enemy_pos[1] + 100 >= 600:
        enemy_direction[1] *= -1

    # Limit player to the window
    if player_pos[0] < 0:
        player_pos[0] = 0
    elif player_pos[0] > 800 - PLAYER_WIDTH:
        player_pos[0] = 800 - PLAYER_WIDTH
    if player_pos[1] < 0:
        player_pos[1] = 0
    elif player_pos[1] > 600 - PLAYER_HEIGHT:
        player_pos[1] = 600 - PLAYER_HEIGHT

    # Animate sprite # CHANGED
    player_index += 0.1
    if player_index >= len(player_sprites):
        player_index = 0        
    player_mask = pygame.mask.from_surface(player_sprites[int(player_index)])

    # Collision detection using masks
    player_rect = pygame.Rect(*player_pos, 50, 50)
    enemy_rect = pygame.Rect(*enemy_pos, 100, 100)
    offset = (enemy_rect.left - player_rect.left, enemy_rect.top - player_rect.top)
    collision = player_mask.overlap(enemy_mask, offset)

    if collision:
        lives -= 1
        collision_sound.play()
        pygame.draw.lines(screen, (255, 0, 0), True, player_outline, 3) # draw red player outline
        screen.blit(font.render("Ganon hit you!", True, (255, 0, 0)), player_pos)
        pygame.display.flip()
        pygame.time.wait(1000)
        start_time += 1000
        # player_pos = [100, 100]  # Reset player position
        player_pos = [random.randint(0,800), random.randint(0,600)]  # respawn player in random position on screen

    # Draw background image
    screen.blit(background_img, (0, 0))
    screen.blit(player_sprites[int(player_index)], player_pos) # CHANGED
    screen.blit(enemy_img, enemy_pos)

    # Draw the outline of the player image
    player_outline = player_mask.outline()
    player_outline = [(x + player_pos[0], y + player_pos[1]) for x, y in player_outline]
    pygame.draw.lines(screen, (0, 255, 0), True, player_outline, 3)

    # Draw the outline of the enemy image
    enemy_outline = enemy_mask.outline()
    enemy_outline = [(x + enemy_pos[0], y + enemy_pos[1]) for x, y in enemy_outline]
    pygame.draw.lines(screen, (0, 255, 0), True, enemy_outline, 3)

    # Display lives and time
    lives_text = font.render(f"Lives: {lives}", True, (0, 0, 0))
    time_text = font.render(f"Time: {remaining_time}s", True, (0, 0, 0))
    screen.blit(lives_text, (10, 10))
    screen.blit(time_text, (10, 50))

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\miche\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
